#### In this example we will look at how to use the K-Nearest_Neighbor algorithm for classification. We will use a modified version of the Video Store data set for this example. We will use the "Incidentals" attribute as the target attribute for classification (the class attribute). The goal is to be able to classify an unseen instance as "Yes" or "No" given the values of "Incidentals" from training instances.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
vstable = pd.read_csv("../data/Video_Store_2.csv", index_col=0)

vstable.shape

In [ ]:
vstable.head()

#### We will be splitting the data into a test and training partions with the test partition to be used for evaluating model error-rate and the training partition to be used to find the K nearest neighbors. Before spliting the data we need to do a random reshuffling to make sure the instances are randomized.

In [ ]:
vs = vstable.reindex(np.random.permutation(vstable.index))
vs.head(10)

In [ ]:
len(vs)

In [ ]:
vs_names = vs.columns.values
vs_names

#### The target attribute for classification is Incidentals:

In [ ]:
vs_target = vs.Incidentals

#### Before we can compute distances we need to convert the data (excluding the target attribute containing the class labels) into standard spreadsheet format with binary dummy variables for categorical attributes).

In [ ]:
vs = pd.get_dummies(vs[['Gender','Income','Age','Rentals','Avg Per Visit','Genre']])
vs.head(10)

#### To be able to evaluate the accuracy of our predictions, we will split the data into training and test sets. In this case, we will use 80% for training and the remaining 20% for testing. Note that we must also do the same split to the target attribute.

In [ ]:
tpercent = 0.8
tsize = int(np.floor(tpercent * len(vs)))
vs_train = vs[:tsize]
vs_test = vs[tsize:]

In [ ]:
print (vs_train.shape)
print (vs_test.shape)

In [ ]:
np.set_printoptions(suppress=True, linewidth=120)

vs_train.head(10)

In [ ]:
vs_test

#### Splitting the target attribute ("Incidentals") accordingly:

In [ ]:
vs_target_train = vs_target[0:int(tsize)]
vs_target_test = vs_target[int(tsize):len(vs)]

In [ ]:
vs_target_train.head()

In [ ]:
vs_target_test

#### Next, we normalize the attributes so that everything is in [0,1] scale. We can use the normalization functions from the kNN module in Ch. 2 of the text. In this case, however, we will use the more flexible and robust scaler function from the preprocessing module of scikit-learn package.

In [ ]:
from sklearn import preprocessing

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(vs_train)

In [ ]:
vs_train_norm = min_max_scaler.fit_transform(vs_train)
vs_test_norm = min_max_scaler.fit_transform(vs_test)

#### Note that while these Scikit-learn functions accept Pandas dataframes as input, they return Numpy arrays (in this case the normalized versions of vs_train and vs_test).

In [ ]:
np.set_printoptions(precision=2, linewidth=100)
vs_train_norm[:10]


#### For consitency, we'll also convert the training and test target labels into Numpy arrays.

In [ ]:
vs_target_train = np.array(vs_target_train)
vs_target_test = np.array(vs_target_test)

In [ ]:
print (vs_target_train)
print ("\n")
print (vs_target_test)

#### The following function illustrates how we can perform a k-nearest-neighbor search. The "measure" argument allows us to use either Euclidean distance or (the inverse of) Cosine similarity as the distance function:

In [ ]:
def knn_search(x, D, K, measure):
    """ find K nearest neighbors of an instance x among the instances in D """
    if measure == 0:
        # euclidean distances from the other points
        dists = np.sqrt(((D - x)**2).sum(axis=1))
    elif measure == 1:
        # first find the vector norm for each instance in D as wel as the norm for vector x
        D_norm = np.array([np.linalg.norm(D[i]) for i in range(len(D))])
        x_norm = np.linalg.norm(x)
        # Compute Cosine: divide the dot product o x and each instance in D by the product of the two norms
        sims = np.dot(D,x)/(D_norm * x_norm)
        # The distance measure will be the inverse of Cosine similarity
        dists = 1 - sims
    idx = np.argsort(dists) # sorting
    # return the indexes of K nearest neighbors
    return idx[:K], dists

In [ ]:
# Let's use vs_test_norm[0] as a test instance x and find its K nearest neighbors
neigh_idx, distances = knn_search(vs_test_norm[0], vs_train_norm, 5, 0)

In [ ]:
vs_test.head(1)

In [ ]:
print (neigh_idx)
print ("\n")
vs_train.iloc[neigh_idx]

In [ ]:
print (distances[neigh_idx])

In [ ]:
neigh_labels = vs_target_train[neigh_idx]
print (neigh_labels)

#### Now that we know the nearest neighbors, we need to find the majority class label among them. The majority class would be the class assgined to the new instance x.

In [ ]:
from collections import Counter
print (Counter(neigh_labels))

In [ ]:
Counter(neigh_labels).most_common(1)

#### Next, we'll use the Knn module from Chapter 2 of Machine Learning in Action. Before importing the whole module, let's illustrate what the code does by stepping through it with some specific input values.

In [ ]:
dataSetSize = vs_train_norm.shape[0]
print (dataSetSize)

In [ ]:
inX = vs_test_norm[0]   # We'll use the first instance in the test data for this example
diffMat = np.tile(inX, (dataSetSize,1)) - vs_train_norm  # Create dataSetSize copies of inX, as rows of a 2D matrix
                                                         # Compute a matrix of differences
print (diffMat[:5,:])

In [ ]:
sqDiffMat = diffMat**2  # The matrix of squared differences
sqDistances = sqDiffMat.sum(axis=1)  # 1D array of the sum of squared differences (one element for each training instance)
distances = sqDistances**0.5  # and finally the matrix of Euclidean distances to inX
print (distances)

In [ ]:
sortedDistIndicies = distances.argsort() # the indices of the training instances in increasing order of distance to inX
print (sortedDistIndicies)

#### To see how the test instance should be classified, we need to find the majority class among the neighbors (here we do not use distance weighting; only a simply voting approach)

In [ ]:
classCount={}
k = 5  # We'll use the top 10 neighbors
for i in range(k):
   voteIlabel = vs_target_train[sortedDistIndicies[i]]
   classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1  # add to the count of the label or retun 1 for first occurrence
   print (sortedDistIndicies[i], voteIlabel, classCount[voteIlabel])


#### Now, let's find the predicted class for the test instance.

In [ ]:
import operator
# Create a dictionary for the class labels with cumulative occurrences across the neighbors as values
# Dictionary will be ordered in decreasing order of the lable values (so the majority class label will
# be the first dictonary element)
sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
print (sortedClassCount)
print (sortedClassCount[0][0])

#### A better way to find the majority class given a list of class labels from neighbors:

In [ ]:
from collections import Counter

k = 5  # We'll use the top 5 neighbors
vote = vs_target_train[sortedDistIndicies[0:k]]
maj_class = Counter(vote).most_common(1)

print (vote)

print (maj_class)

print ("Class label for the classified istance: ", maj_class[0][0])

#### Let's know import the whole KNN module from MLA (uploaded on D2L) and use as part of a more robust evaluation process. We will step through all test instances, use our Knn classifier to predict a class label for each instance, and in each case we compare the predicted label to the actual value from the target test labels.

In [ ]:
import kNN

In [ ]:
numTestVecs = len(vs_target_test)
print (numTestVecs)

In [ ]:
errorCount = 0.0
for i in range(numTestVecs):
    # classify0 function uses Euclidean distance to find k nearest neighbors
    classifierResult = kNN.classify0(vs_test_norm[i,:], vs_train_norm, vs_target_train, 3)
    print ("the classifier came back with: %s, the real answer is: %s" % (classifierResult, vs_target_test[i]))
    if (classifierResult != vs_target_test[i]): errorCount += 1.0
        
print ("the total error rate is: %f" % (errorCount/float(numTestVecs)))

#### I have added a new classifier function to the kNN module that uses Cosine similarity instead of Euclidean distance:

In [ ]:
from importlib import reload
reload(kNN)

In [ ]:
errorCount = 0.0
for i in range(numTestVecs):
    # classify1 function uses inverse of Cosine similarity to find k nearest neighbors
    classifierResult2 = kNN.classify1(vs_test_norm[i,:], vs_train_norm, vs_target_train, 3)
    print ("the classifier came back with: %s, the real answer is: %s" % (classifierResult2, vs_target_test[i]))
    if (classifierResult2 != vs_target_test[i]): errorCount += 1.0
print ("the total error rate is: %f" % (errorCount/float(numTestVecs)))